In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import os
import scipy.optimize as opt
from scipy.optimize import LinearConstraint, Bounds

In [3]:
#reading in orlin's cleaned csv data
local_filepath = "/Users/leonlu/Downloads/gold_data.csv" # TODO: I can't import gold_data.csv as a dataframe
gold_data = pd.read_csv(local_filepath, index_col=0)
display(gold_data.head())

,body,html{height:100%;width:100%}.button{border:none;cursor:pointer;color:#0b57d0;font-family:Google Sans Text,Roboto,sans-serif;font-size:14px;border-radius:100px;padding:12px;margin:0 8px;text-decoration:none}.button.primary-button{background-color:#0b57d0;color:#fff;padding:12px 24px}.icon{height:48px}.container{width:800px;height:212px;-webkit-box-orient:vertical;-webkit-box-direction:normal;-webkit-flex-direction:column;-moz-box-orient:vertical;-moz-box-direction:normal;-ms-flex-direction:column;flex-direction:column;display:-webkit-box;display:-webkit-flex;display:-moz-box;display:-ms-flexbox;display:flex;-webkit-box-align:center;-webkit-align-items:center;-moz-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-pack:justify;-webkit-justify-content:space-between;-moz-box-pack:justify;-ms-flex-pack:justify;justify-content:space-between}.error,.login,.request-storage-access{display:none}.error,.login.1,.request-storage-access,.too-many-login-redirects{height:100%;width:100%;-webkit-box-align:center;-webkit-align-items:center;-moz-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-pack:center;-webkit-justify-content:center;-moz-box-pack:center;-ms-flex-pack:center;justify-content:center}.document-root.show-error .error,.document-root.show-login-page .login,.document-root.show-storage-access .request-storage-access,.too-many-login-redirects{display:-webkit-box;display:-webkit-flex;display:-moz-box;display:-ms-flexbox;display:flex}.title-text{font-family:Google Sans,Roboto.1,sans-serif;font-size:28px;line-height:36px}.subtitle-text{font-family:Google Sans Text,Roboto.2,sans-serif;font-size:16px;line-height:24px}
"<!DOCTYPE html><style nonce=""94egGagWic13rgAsNPwrgQ"">.document-root",,,,,,,,,,,,,,,,
"/*# sourceMappingURL=style.css.map */</style><script nonce=""O98UMmcjg0SVFXbXzwi10w"">'use strict';var e=[""storage_access_granted""",not_in_iframe,login_counter];function f(a,b,"c){c=void 0===c?""true"":c;a=new URL(a);for(var ...",c);return a.toString()};/*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Copyright The Closure Library Authors.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SPDX-License-Identifier: Apache-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
*/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"function h(){var a=document.getElementsByClassName(""document-root"")[0]",b=this;this.g=new l;this.h=a;a.getElementsByCl...,function(){return void m(b)});a.getElementsByC...,function(c){return void n(c)})},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#construct reward matrix R_{i,t} of dimension (K x N)
reward_df = gold_data.iloc[:, -6:].pct_change()
reward_df.dropna(how = 'all', inplace = True)
reward_df.drop(reward_df.index[-1], inplace=True) #drops the last line of data
display(reward_df)

IndexError: index -1 is out of bounds for axis 0 with size 0

### Implementation of Huo and Fu begins here

In [26]:
K = 6 #number of peripheral assets (non AUSHF) in market
gamma_ = 0.95 #confidence level for CVaR performance function F
delta = 5
lambda_ = np.random.uniform(0, 1) # risk preference (->1 means UCB1, ->0 means min CVaR)
time_horizon = 100 #number of time-steps/episodes
times_selected = np.zeros(K)

reward_array = np.random.uniform(size=(K, time_horizon))
H_matrix = reward_array[:, :delta] # temporary definition; change to historical_returns
R_matrix = reward_array[:, delta:]

display(reward_array[:, :5])
print(reward_array.shape)
display(H_matrix[:, :5])
print(H_matrix.shape)
display(R_matrix[:, :5])
print(R_matrix.shape)

array([[0.83562885, 0.1603224 , 0.63073708, 0.21860645, 0.11366412],
       [0.93255255, 0.85293328, 0.78836682, 0.35810665, 0.86430021],
       [0.64334673, 0.11188826, 0.28615242, 0.95398195, 0.29110134],
       [0.41529982, 0.54590975, 0.06320164, 0.85266938, 0.69555826],
       [0.82509283, 0.08867013, 0.87890779, 0.91627397, 0.00411855],
       [0.39538784, 0.30098958, 0.81202646, 0.46031973, 0.45996191]])

(6, 100)


array([[0.83562885, 0.1603224 , 0.63073708, 0.21860645, 0.11366412],
       [0.93255255, 0.85293328, 0.78836682, 0.35810665, 0.86430021],
       [0.64334673, 0.11188826, 0.28615242, 0.95398195, 0.29110134],
       [0.41529982, 0.54590975, 0.06320164, 0.85266938, 0.69555826],
       [0.82509283, 0.08867013, 0.87890779, 0.91627397, 0.00411855],
       [0.39538784, 0.30098958, 0.81202646, 0.46031973, 0.45996191]])

(6, 5)


array([[2.60821586e-01, 1.39441468e-02, 6.26972583e-01, 4.68496142e-01,
        5.45310265e-01],
       [9.30864427e-01, 3.62652697e-01, 2.29438634e-01, 8.57005694e-02,
        1.73493250e-01],
       [8.11894163e-02, 2.40372533e-01, 3.12360352e-01, 6.48083136e-02,
        9.96461729e-01],
       [7.46812878e-01, 5.08334122e-01, 7.37279599e-01, 3.94804197e-01,
        6.87818057e-01],
       [6.74682143e-01, 7.77058832e-01, 8.36377368e-01, 4.75611851e-01,
        4.73088132e-01],
       [7.26159784e-01, 2.13993517e-01, 4.55379084e-01, 1.84085727e-04,
        8.48644468e-01]])

(6, 95)


In [27]:
# define helper functions used in huo/fu paper
    
def I(t):
    """
    Definition: Equation 2.1
    """
    if t < K: # strict inequality due to assets being 0, 1, ..., K-1
        selection = t
    else:
        R_bar = R_matrix.mean(axis=1) # take the mean across the time axis
        selection = np.argmax([R_bar[i] + np.sqrt((2 * np.log(t) / times_selected[i] * (t - 1))) for i in range(K)])
    times_selected[selection] += 1
    return selection


# NOTE: not necessary; kept for safety      
# def F(gamma, u, alpha, t, delta):
#     """
#     Definition: Equation 2.3; estimates the conditional value-at-risk.
#     H_matrix: the s-th column of H_matrix (K rows, \delta columns) is the historical returns of the s-th asset
#     R_matrix: the s-th column of R_matrix (K rows, t - 1 columns) is the trial of returns observed so far of the s-th asset
#     """
#     sum1 = np.sum([np.max(-np.dot(H_matrix.iloc[:, s], u) - alpha, 0) for s in range(delta)])
#     sum2 = np.sum([np.max(-np.dot(R_matrix.iloc[:, s], u) - alpha, 0) for s in range(t - 1)])
#     return alpha + (sum1 + sum2) / ((delta + t - 1) * (1 - gamma))

In [69]:
# computation of different omega components
def omega_C(t):
    """
    Definition: Equation 2.4
    Computes a risk-aware portfolio according to Equation 2.3.
    t: the time
    """
    # the objective function at iteration t
    def F_t(args):
        """
        Definition: Equation 2.3; estimates the conditional value-at-risk.
        H_matrix: the s-th column of H_matrix (K rows, \delta columns) is the historical returns of the s-th asset
        R_matrix: the s-th column of R_matrix (K rows, t - 1 columns) is the trial of returns observed so far of the s-th asset
        """
        u, alpha = args[:-1], args[-1]
        H_sum = np.sum([np.max(-np.dot(H_matrix[:, s], u) - alpha, 0) for s in range(delta)])
        R_sum = np.sum([np.max(-np.dot(R_matrix[:, s], u) - alpha, 0) for s in range(t - 1)])
        print(H_sum, R_sum)
        return alpha + (H_sum + R_sum) / ((delta + t - 1) * (1 - gamma_))

    # constraining u to be a probability distribution over 1, 2, ..., K
    def probability_constraint(u):
        return np.sum(u[:K]) - 1
    bounds = [(0, 1)] * K + [(None, None)]

    # scipy.optimize.minimize
    x0 = np.ones(K + 1)
    x0[:K] /= K 

    result = opt.minimize(
        F_t, x0 = x0,
        method = 'Powell',
        constraints = [{'type': 'eq', 'fun': probability_constraint}],
        bounds = bounds
    )
    if not result['success']:
        print('Warning: minimization failed. Reason: {}'.format(result['message']))
    approx_min = result['x'][:K]
    return approx_min

In [70]:
def sequential_selection_algo(K, lambda_, gamma=gamma_, time_horizon=time_horizon):
    """
    K: number of assets
    gamma: confidence level
    lambda_: risk preference
    """
    returns = []
    for t in range(time_horizon):
        # Equation. 2.2: compute omega_M(t)
        omega_M = np.zeros(K) 
        omega_M[I(t)] = 1
        
        # Equation 2.4: compute the risk aware portfolio
        risk_aware_portfolio = omega_C(t)

        # Equation 2.5: compute convex combination of UCB1 portfolio and risk aware portfolio
        print(risk_aware_portfolio.round(3))
        omega_star = lambda_ * omega_M + (1 - lambda_) * risk_aware_portfolio

        # "Receive portfolio reward"
        returns_by_t = np.dot(omega_star, R_matrix[:, t])
        returns.append(returns_by_t)
    return returns # idk what to return; there might be a formula somewhere

In [71]:
sequential_selection_algo(K, lambda_, gamma=gamma_, time_horizon=time_horizon)

/Users/popcornfury/miniforge3/envs/dsc80/lib/python3.8/site-packages/scipy/optimize/_minimize.py:561: RuntimeWarning: Method Powell cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,
/Users/popcornfury/miniforge3/envs/dsc80/lib/python3.8/site-packages/scipy/optimize/_optimize.py:2787: RuntimeWarning: overflow encountered in double_scalars
  if (w - xc) * (xb - w) > 0.0:
/Users/popcornfury/miniforge3/envs/dsc80/lib/python3.8/site-packages/scipy/optimize/_optimize.py:2803: RuntimeWarning: overflow encountered in double_scalars
  elif (w - wlim)*(wlim - xc) >= 0.0:
/Users/popcornfury/miniforge3/envs/dsc80/lib/python3.8/site-packages/scipy/optimize/_optimize.py:2782: RuntimeWarning: overflow encountered in double_scalars
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom
/Users/popcornfury/miniforge3/envs/dsc80/lib/python3.8/site-packages/scipy/optimize/_optimize.py:2775: RuntimeWarning: overflow encountered in double_scalars
  tmp1 = (xb - xa) * (fb -

-7.626012790504615 0.0
-8.047775357206312 0.0
-8.510222822148798 0.0
-8.796031073494479 0.0
-8.972670287091285 0.0
-9.08183932484016 0.0
-9.149309500688087 0.0
-9.191008362589038 0.0
-9.216779676536014 0.0
-9.232707224489989 0.0
-9.24255099048299 0.0
-9.248634772443964 0.0
-9.25239475647599 0.0
-9.254718554404938 0.0
-9.256154740508016 0.0
-9.257042352333889 0.0
-9.257590926611094 0.0
-9.257929964159759 0.0
-9.258139500888298 0.0
-9.258269001708426 0.0
-9.258349037616837 0.0
-10.075681221593431 0.0
-10.971856525750312 0.0
-11.52572332359754 0.0
-11.868031829907194 0.0
-12.079590121444769 0.0
-12.210340336216849 0.0
-12.291148412982341 0.0
-12.341090550988927 0.0
-12.371956489747836 0.0
-12.391032688995516 0.0
-12.402822428506745 0.0
-12.410108888243194 0.0
-12.414612168017975 0.0
-12.417395347979639 0.0
-12.419115447792757 0.0
-12.420178527941308 0.0
-12.420835547605874 0.0
-12.421241608089854 0.0
-12.421492567270443 0.0
-12.421647668573836 0.0
-12.913923313163588 0.0
-13.4536858285374

/Users/popcornfury/miniforge3/envs/dsc80/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


-2.450933783673034e+262 -6.862614594284496e+262
-3.965694193731613e+262 -1.1103943742448516e+263
-6.416628039060339e+262 -1.796655850936895e+263
-1.0382322332552955e+263 -2.907050253114827e+263
-1.6798950533029988e+263 -4.7037061492483965e+263
-2.7181273126760646e+263 -7.610756475492979e+263
-4.3980224082385033e+263 -1.2314462743067808e+264
-7.116149789291778e+263 -1.9925219410016972e+264
-1.1514172308166931e+264 -3.223968246286741e+264
-1.8630322276472568e+264 -5.216490237412319e+264
-3.0144494874290016e+264 -8.440458564801207e+264
-4.877481761942696e+264 -1.3656948933439548e+265
-7.8919313252031885e+264 -2.2097407710568922e+265
-1.2769413209843817e+265 -3.5754356987562684e+265
-2.066134473357643e+265 -5.7851765254014006e+265
-3.34307582646476e+265 -9.360612314101326e+265
-5.4092103517980825e+265 -1.514578898503463e+266
-8.752286262361259e+265 -2.4506401534611527e+266
-1.4161496750233436e+266 -3.9652190900653623e+266
-2.291378323276721e+266 -6.415859305174818e+266
-3.7075280339247254e

KeyboardInterrupt: 